### Importación de librerías y funciones

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from funciones import *

### Importación de datos de eventos de partidos. 
#### 4 partidos de España en el mundial: ESP-CRI, ESP-ALE, ESP-JAP Y ESP-MAR

In [2]:
df_esp_cri=pd.read_csv(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\myairbridge-mktRRgViKIY\DATOS 221209\20221123_FWC_10_ESP-CRC_FIFA_Unified_Event_Data_Events.csv')
df_esp_ale=pd.read_csv(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\myairbridge-mktRRgViKIY\DATOS 221209\20221127_FWC_28_ESP-GER_FIFA_Unified_Event_Data_Events (1).csv')
df_esp_jap=pd.read_csv(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\myairbridge-mktRRgViKIY\DATOS 221209\20221201_FWC_43_JPN-ESP_FIFA_Unified_Event_Data_Events.csv')
df_esp_mar=pd.read_csv(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\myairbridge-mktRRgViKIY\DATOS 221209\20221206_FWC_55_MAR-ESP_FIFA_Unified_Event_Data_Events.csv')

### Transformación inicial y concatenación 

In [3]:
lista_eventos_partidos = [df_esp_cri, df_esp_ale, df_esp_jap, df_esp_mar]

In [4]:
for i in lista_eventos_partidos:
    i['match_time_in_min'] = i.match_time_in_ms.apply(pasar_ms_a_min)

In [5]:
df_ev = pd.concat([df_esp_cri, df_esp_ale, df_esp_jap, df_esp_mar], axis=0)

In [6]:
#df_ev.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\datos_para_sql\tabla_eventos.xlsx', index=False)

In [7]:
#df_ev.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\viz\ficheros Tableau\tabla_eventos.xlsx', index=False)

### CÁLCULO DE MÉTRICAS

### Cálculo de métricas de eventos de los partidos

In [8]:
est_esp_cri = metricas_partido(df_esp_cri)
est_esp_ale = metricas_partido(df_esp_ale)
est_esp_jap = metricas_partido(df_esp_jap)
est_esp_mar = metricas_partido(df_esp_mar)

In [9]:
df_est = pd.concat([est_esp_cri, est_esp_ale, est_esp_jap, est_esp_mar], axis=0)
df_est

,match_id,team,corners,faltas_lanzadas,faltas_directas_lanzadas,faltas_realizadas,penaltis,penaltis_marcados,%_acierto_penaltis,saques_banda,saques_banda_recepcionados,%_acierto_sb,ofrecimientos,ofrecimientos_con_movimiento,ofrecimientos_con_movimiento_camp_contr,ofrecimientos_con_movimiento_ult_terc,remates_totales,remates_totales_fa,remates_on_target,remates_on_target_fa,remates_off_target,%_remates_on_target,goles,goles_fa,%_gol/on_target,goles_concedidos,centros_totales,centros_completados,asistencias_centro,pases,pases_completados,%_acierto_pases,pases_completados_linebreak,asistencias_pase,cambios_de_juego,pases_ultimo_tercio,pases_completados_ultimo_tercio,progresiones,intentos_regate,intentos_regate_ult_terc,regates_completados,regates_completados_ult_terc,recepciones,recep_behind,recep_3_4,recep_2_3,recep_1_2,recep_inside,%_recep_inside,recep_outside,%_recep_outside,recep_ultimo_tercio,bloqueos,despejes,intercepciones,entradas_totales,entradas_exitosas,%_entradas_exitosas,take_on_against,pushing-on,presiones,presiones_directas,%_presion_directa,gk_active_eng,gk_aerial_control,gk_def_support,gk_goalprevention_tot,gk_goalprevention_tot_saved,duelos_totales,duelos_totales_ganados,%_duelos_totales_ganados,duelos_campo_ganados,duelos_físicos_ganados,duelos_aereos_ganados,sustituciones,posesion,posesión_disputa,pases_verticales,pases_adelante,pases_atras,pases_horizontales,%_pases_verticales,%_pases_adelante,%_pases_atras,%_pases_horizontales,recuperaciones,recuperaciones_campo_contr,%_recuperaciones_campo_contr,recuperaciones_campo_prop,%_recuperaciones_campo_prop,recuperaciones1,recuperaciones_campo_contr1,%_recuperaciones_campo_contr1,recuperaciones_campo_prop1,%_recuperaciones_campo_prop1,perdidas,perdidas_campo_prop,%_perdidas_campo_prop,tarj_amarillas,tarjetas_rojas
0,133004,SPAIN,5,18,12,11,1,1,100.0,24,24,100.0,1677,1085,694,309,17,3,7,0,8,41.2,7,0,100.0,0,17,2,2,1037,976,94.1,193,2,8,203,180,42,11,9,2,2,1092,23,0,60,307,628,57.5,464,42.5,232,16,10,4,18,7,38.9,4,208,185,50,27.0,3,1,1,0,0,37,19,51,6,2,11,5,78.81,1.96,87,153,136,687,9,16,14,70,38,18,47,20,53,47,20,43,27,57,116,26,22,0,0
1,133004,COSTA RICA,0,11,8,18,0,0,none,13,11,84.6,409,250,103,43,0,0,0,0,8,0.0,0,0,0.0,7,5,0,0,218,159,72.9,43,0,1,21,16,9,4,2,2,1,226,1,1,23,74,139,61.5,87,38.5,31,21,30,8,42,13,31.0,11,795,585,47,8.0,7,0,7,17,10,37,18,49,1,1,16,5,19.23,1.96,22,37,27,95,14,23,17,60,38,7,18,31,82,39,7,18,32,82,104,42,40,2,0
0,133021,SPAIN,6,16,14,15,0,0,none,23,21,91.3,1040,633,282,112,7,2,3,1,2,42.9,1,0,33.3,1,9,2,1,624,542,86.9,118,0,9,69,55,31,11,5,1,0,666,16,3,47,186,362,54.4,304,45.6,92,24,28,6,40,13,32.5,9,316,275,45,16.4,5,4,3,11,9,33,16,48,6,1,9,4,60.23,2.17,53,98,88,356,10,18,16,66,47,13,28,34,72,50,15,30,35,70,139,62,45,1,0
1,133021,GERMANY,5,15,13,16,0,0,none,15,8,53.3,670,440,209,107,10,3,4,0,2,40.0,1,0,25.0,1,13,1,0,337,270,80.1,63,0,2,53,30,30,9,4,4,1,379,9,2,34,132,246,64.9,133,35.1,82,32,24,8,45,14,31.1,11,508,387,59,15.2,3,0,2,7,6,36,18,50,8,1,9,5,37.60,2.17,35,59,42,169,13,22,16,63,48,15,31,33,69,53,16,30,37,70,128,32,25,3,0
0,133035,SPAIN,2,11,9,8,0,0,none,22,20,90.9,1747,1138,825,451,14,7,5,1,2,35.7,1,0,20.0,2,20,2,0,1048,969,92.5,126,1,8,244,218,25,10,6,6,3,1092,17,0,18,193,674,61.7,418,38.3,282,7,15,11,16,8,50.0,4,173,150,25,16.7,1,0,1,6,4,30,13,43,3,0,10,5,78.95,1.58,59,123,123,723,6,13,13,75,35,11,31,24,69,35,11,31,24,69,128,19,15,0,0
1,133035,JAPAN,0,8,6,11,0,0,none,11,10,90.9,437,315,123,56,6,2,3,1,2,50.0,2,1,66.7,1,6,3,1,214,155,72.4,45,1,2,29,26,10,4,2,2,1,237,10,0,21,75,159,67.1,78,32.9,44,39,39,10,29,6,20.7,10,791,637,62,9.7,3,1,1,14,13,30,17,57,4,1,12,5,19.47,1.58,27,44,20,91,17,28,13,59,34,4,12,30,88,34,4,12,30,88,91,34,37,3,0
0,128074,SPAIN,8,20,14,18,3,0,0.0,33,33,100.0,1621,896,779,446,16,3,3,0,6,18.8,0,0,0.0,3,27,4,0,1008,932,92.5,126,0,8,196,167,26,13,8,4,1,1075,27,0,26,165,697,64.8,378,35.2,245,19,26,7,41,5,12.2,11,222,217,56,25.8,1,1,0,10,6,47,23,49,4,2,17,6,71.35,2.31,96,142,79,711,10,15,8,76,33,12,36,21,64,36,13

In [10]:
df_est['asistencias_totales'] = df_est.asistencias_centro + df_est.asistencias_pase

In [11]:
xG = [3.5, 0, 0.6, 1.3, 1.4, 1, 0.7, 1]
df_est['xG'] = xG

In [12]:
df_est

,match_id,team,corners,faltas_lanzadas,faltas_directas_lanzadas,faltas_realizadas,penaltis,penaltis_marcados,%_acierto_penaltis,saques_banda,saques_banda_recepcionados,%_acierto_sb,ofrecimientos,ofrecimientos_con_movimiento,ofrecimientos_con_movimiento_camp_contr,ofrecimientos_con_movimiento_ult_terc,remates_totales,remates_totales_fa,remates_on_target,remates_on_target_fa,remates_off_target,%_remates_on_target,goles,goles_fa,%_gol/on_target,goles_concedidos,centros_totales,centros_completados,asistencias_centro,pases,pases_completados,%_acierto_pases,pases_completados_linebreak,asistencias_pase,cambios_de_juego,pases_ultimo_tercio,pases_completados_ultimo_tercio,progresiones,intentos_regate,intentos_regate_ult_terc,regates_completados,regates_completados_ult_terc,recepciones,recep_behind,recep_3_4,recep_2_3,recep_1_2,recep_inside,%_recep_inside,recep_outside,%_recep_outside,recep_ultimo_tercio,bloqueos,despejes,intercepciones,entradas_totales,entradas_exitosas,%_entradas_exitosas,take_on_against,pushing-on,presiones,presiones_directas,%_presion_directa,gk_active_eng,gk_aerial_control,gk_def_support,gk_goalprevention_tot,gk_goalprevention_tot_saved,duelos_totales,duelos_totales_ganados,%_duelos_totales_ganados,duelos_campo_ganados,duelos_físicos_ganados,duelos_aereos_ganados,sustituciones,posesion,posesión_disputa,pases_verticales,pases_adelante,pases_atras,pases_horizontales,%_pases_verticales,%_pases_adelante,%_pases_atras,%_pases_horizontales,recuperaciones,recuperaciones_campo_contr,%_recuperaciones_campo_contr,recuperaciones_campo_prop,%_recuperaciones_campo_prop,recuperaciones1,recuperaciones_campo_contr1,%_recuperaciones_campo_contr1,recuperaciones_campo_prop1,%_recuperaciones_campo_prop1,perdidas,perdidas_campo_prop,%_perdidas_campo_prop,tarj_amarillas,tarjetas_rojas,asistencias_totales,xG
0,133004,SPAIN,5,18,12,11,1,1,100.0,24,24,100.0,1677,1085,694,309,17,3,7,0,8,41.2,7,0,100.0,0,17,2,2,1037,976,94.1,193,2,8,203,180,42,11,9,2,2,1092,23,0,60,307,628,57.5,464,42.5,232,16,10,4,18,7,38.9,4,208,185,50,27.0,3,1,1,0,0,37,19,51,6,2,11,5,78.81,1.96,87,153,136,687,9,16,14,70,38,18,47,20,53,47,20,43,27,57,116,26,22,0,0,4,3.5
1,133004,COSTA RICA,0,11,8,18,0,0,none,13,11,84.6,409,250,103,43,0,0,0,0,8,0.0,0,0,0.0,7,5,0,0,218,159,72.9,43,0,1,21,16,9,4,2,2,1,226,1,1,23,74,139,61.5,87,38.5,31,21,30,8,42,13,31.0,11,795,585,47,8.0,7,0,7,17,10,37,18,49,1,1,16,5,19.23,1.96,22,37,27,95,14,23,17,60,38,7,18,31,82,39,7,18,32,82,104,42,40,2,0,0,0.0
0,133021,SPAIN,6,16,14,15,0,0,none,23,21,91.3,1040,633,282,112,7,2,3,1,2,42.9,1,0,33.3,1,9,2,1,624,542,86.9,118,0,9,69,55,31,11,5,1,0,666,16,3,47,186,362,54.4,304,45.6,92,24,28,6,40,13,32.5,9,316,275,45,16.4,5,4,3,11,9,33,16,48,6,1,9,4,60.23,2.17,53,98,88,356,10,18,16,66,47,13,28,34,72,50,15,30,35,70,139,62,45,1,0,1,0.6
1,133021,GERMANY,5,15,13,16,0,0,none,15,8,53.3,670,440,209,107,10,3,4,0,2,40.0,1,0,25.0,1,13,1,0,337,270,80.1,63,0,2,53,30,30,9,4,4,1,379,9,2,34,132,246,64.9,133,35.1,82,32,24,8,45,14,31.1,11,508,387,59,15.2,3,0,2,7,6,36,18,50,8,1,9,5,37.60,2.17,35,59,42,169,13,22,16,63,48,15,31,33,69,53,16,30,37,70,128,32,25,3,0,0,1.3
0,133035,SPAIN,2,11,9,8,0,0,none,22,20,90.9,1747,1138,825,451,14,7,5,1,2,35.7,1,0,20.0,2,20,2,0,1048,969,92.5,126,1,8,244,218,25,10,6,6,3,1092,17,0,18,193,674,61.7,418,38.3,282,7,15,11,16,8,50.0,4,173,150,25,16.7,1,0,1,6,4,30,13,43,3,0,10,5,78.95,1.58,59,123,123,723,6,13,13,75,35,11,31,24,69,35,11,31,24,69,128,19,15,0,0,1,1.4
1,133035,JAPAN,0,8,6,11,0,0,none,11,10,90.9,437,315,123,56,6,2,3,1,2,50.0,2,1,66.7,1,6,3,1,214,155,72.4,45,1,2,29,26,10,4,2,2,1,237,10,0,21,75,159,67.1,78,32.9,44,39,39,10,29,6,20.7,10,791,637,62,9.7,3,1,1,14,13,30,17,57,4,1,12,5,19.47,1.58,27,44,20,91,17,28,13,59,34,4,12,30,88,34,4,12,30,88,91,34,37,3,0,2,1.0
0,128074,SPAIN,8,20,14,18,3,0,0.0,33,33,100.0,1621,896,779,446,16,3,3,0,6,18.8,0,0,0.0,3,27,4,0,1008,932,92.5,126,0,8,196,167,26,13,8,4,1,1075,27,0,26,165,697,64.8,378,35.2,245,19,26,7,41,5,12.2,11,222,217,56,25.8,1,1,0,10,6,47,23,49,4,2,17

### Importación de los datos de tracking generales

In [13]:
df_tr_esp_cri = pd.read_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\myairbridge-mktRRgViKIY\DATOS 221209\Tracking Data Metrics M-10.xlsx')
df_tr_esp_ale = pd.read_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\myairbridge-mktRRgViKIY\DATOS 221209\Tracking Data Metrics M-28 (1).xlsx')
df_tr_esp_jap = pd.read_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\myairbridge-mktRRgViKIY\DATOS 221209\Tracking Data Metrics M-43.xlsx')
df_tr_esp_mar = pd.read_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\myairbridge-mktRRgViKIY\DATOS 221209\Tracking Data Metrics M-55.xlsx')

### Cálculo de métricas de tracking

In [14]:
est_tr_esp_cri = metricas_tracking_partido(df_tr_esp_cri)
est_tr_esp_ale = metricas_tracking_partido(df_tr_esp_ale)
est_tr_esp_jap = metricas_tracking_partido(df_tr_esp_jap)
est_tr_esp_mar = metricas_tracking_partido(df_tr_esp_mar)

### Concatenación de tablas con las métricas de tracking

In [15]:
df_est_tr = pd.concat([est_tr_esp_cri, est_tr_esp_ale, est_tr_esp_jap, est_tr_esp_mar], axis=0)

In [16]:
#df_est.team = df_est.team.apply(capitalizar)

### Unión lateral de métricas de eventos y métricas de tracking

In [17]:
df_est = df_est.reset_index(drop=True)
df_est_tr = df_est_tr.reset_index(drop=True)

In [18]:
df_est['distancia_recorrida_km'] = df_est_tr['distancia_recorrida_km']
df_est['n_speed_runs_(15-20km/h)'] = df_est_tr['n_speed_runs_(15-20km/h)']
df_est['n_of_sprints_(>20km/h)'] = df_est_tr['n_of_sprints_(>20km/h)']
df_est['total_average_speed_(km/h)'] = df_est_tr['total_average_speed_(km/h)']

In [19]:
#df_est.set_index('team').join(df_est_tr.set_index('team'))

In [20]:
#pd.merge(df_est, df_est_tr, how='left', on=['team', 'team'])

### Tabla final de métricas calculadas

In [21]:
df_est

,match_id,team,corners,faltas_lanzadas,faltas_directas_lanzadas,faltas_realizadas,penaltis,penaltis_marcados,%_acierto_penaltis,saques_banda,saques_banda_recepcionados,%_acierto_sb,ofrecimientos,ofrecimientos_con_movimiento,ofrecimientos_con_movimiento_camp_contr,ofrecimientos_con_movimiento_ult_terc,remates_totales,remates_totales_fa,remates_on_target,remates_on_target_fa,remates_off_target,%_remates_on_target,goles,goles_fa,%_gol/on_target,goles_concedidos,centros_totales,centros_completados,asistencias_centro,pases,pases_completados,%_acierto_pases,pases_completados_linebreak,asistencias_pase,cambios_de_juego,pases_ultimo_tercio,pases_completados_ultimo_tercio,progresiones,intentos_regate,intentos_regate_ult_terc,regates_completados,regates_completados_ult_terc,recepciones,recep_behind,recep_3_4,recep_2_3,recep_1_2,recep_inside,%_recep_inside,recep_outside,%_recep_outside,recep_ultimo_tercio,bloqueos,despejes,intercepciones,entradas_totales,entradas_exitosas,%_entradas_exitosas,take_on_against,pushing-on,presiones,presiones_directas,%_presion_directa,gk_active_eng,gk_aerial_control,gk_def_support,gk_goalprevention_tot,gk_goalprevention_tot_saved,duelos_totales,duelos_totales_ganados,%_duelos_totales_ganados,duelos_campo_ganados,duelos_físicos_ganados,duelos_aereos_ganados,sustituciones,posesion,posesión_disputa,pases_verticales,pases_adelante,pases_atras,pases_horizontales,%_pases_verticales,%_pases_adelante,%_pases_atras,%_pases_horizontales,recuperaciones,recuperaciones_campo_contr,%_recuperaciones_campo_contr,recuperaciones_campo_prop,%_recuperaciones_campo_prop,recuperaciones1,recuperaciones_campo_contr1,%_recuperaciones_campo_contr1,recuperaciones_campo_prop1,%_recuperaciones_campo_prop1,perdidas,perdidas_campo_prop,%_perdidas_campo_prop,tarj_amarillas,tarjetas_rojas,asistencias_totales,xG,distancia_recorrida_km,n_speed_runs_(15-20km/h),n_of_sprints_(>20km/h),total_average_speed_(km/h)
0,133004,SPAIN,5,18,12,11,1,1,100.0,24,24,100.0,1677,1085,694,309,17,3,7,0,8,41.2,7,0,100.0,0,17,2,2,1037,976,94.1,193,2,8,203,180,42,11,9,2,2,1092,23,0,60,307,628,57.5,464,42.5,232,16,10,4,18,7,38.9,4,208,185,50,27.0,3,1,1,0,0,37,19,51,6,2,11,5,78.81,1.96,87,153,136,687,9,16,14,70,38,18,47,20,53,47,20,43,27,57,116,26,22,0,0,4,3.5,122.43399,1331,548,6.720440
1,133004,COSTA RICA,0,11,8,18,0,0,none,13,11,84.6,409,250,103,43,0,0,0,0,8,0.0,0,0,0.0,7,5,0,0,218,159,72.9,43,0,1,21,16,9,4,2,2,1,226,1,1,23,74,139,61.5,87,38.5,31,21,30,8,42,13,31.0,11,795,585,47,8.0,7,0,7,17,10,37,18,49,1,1,16,5,19.23,1.96,22,37,27,95,14,23,17,60,38,7,18,31,82,39,7,18,32,82,104,42,40,2,0,0,0.0,116.61037,1455,453,6.505847
2,133021,SPAIN,6,16,14,15,0,0,none,23,21,91.3,1040,633,282,112,7,2,3,1,2,42.9,1,0,33.3,1,9,2,1,624,542,86.9,118,0,9,69,55,31,11,5,1,0,666,16,3,47,186,362,54.4,304,45.6,92,24,28,6,40,13,32.5,9,316,275,45,16.4,5,4,3,11,9,33,16,48,6,1,9,4,60.23,2.17,53,98,88,356,10,18,16,66,47,13,28,34,72,50,15,30,35,70,139,62,45,1,0,1,0.6,112.65189,1461,557,6.504642
3,133021,GERMANY,5,15,13,16,0,0,none,15,8,53.3,670,440,209,107,10,3,4,0,2,40.0,1,0,25.0,1,13,1,0,337,270,80.1,63,0,2,53,30,30,9,4,4,1,379,9,2,34,132,246,64.9,133,35.1,82,32,24,8,45,14,31.1,11,508,387,59,15.2,3,0,2,7,6,36,18,50,8,1,9,5,37.60,2.17,35,59,42,169,13,22,16,63,48,15,31,33,69,53,16,30,37,70,128,32,25,3,0,0,1.3,115.54569,1559,617,6.681131
4,133035,SPAIN,2,11,9,8,0,0,none,22,20,90.9,1747,1138,825,451,14,7,5,1,2,35.7,1,0,20.0,2,20,2,0,1048,969,92.5,126,1,8,244,218,25,10,6,6,3,1092,17,0,18,193,674,61.7,418,38.3,282,7,15,11,16,8,50.0,4,173,150,25,16.7,1,0,1,6,4,30,13,43,3,0,10,5,78.95,1.58,59,123,123,723,6,13,13,75,35,11,31,24,69,35,11,31,24,69,128,19,15,0,0,1,1.4,114.04544,1326,459,6.558801
5,133035,JAPAN,0,8,6,11,0,0,none,11,10,90.9,437,315,123,56,6,2,3,1,2,50.0,2,1,66.7,1,6,3,1,214,155,72.4,45,1,2,29,26,10,4,2,2,1,237,10,0,21,75,159,67.1,78,32.9,44,39,39,10,29,6,20.7,10,791,637,62,9.7,3,1,1,14,13,30,17,57,4,1,12,5,19.47,1.58,27,44,20,91,17,28,13,59,34,4,12,30,88,34,4,12,30,88,91,34,37,3,0,2,1.0,11

In [22]:
#df_est.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\datos_para_sql\tabla_metricas.xlsx', index=False)

In [23]:
#df_est.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\viz\ficheros Tableau\tabla_metricas.xlsx', index=False)

### MINUTOS JUGADOS DE CADA JUGADOR

### Importación de tabla jugadores y preparación de la misma

In [24]:
df_pl=pd.read_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\pruebas\players.xlsx')
df_pl.head()

,competition_id,team_id,team,player_id,player_shirt_number,player_position,player_name,player_first_name,player_last_name,player_date_birth,team_code
0,255711,43901,Costa Rica,199175,1,goalkeeper,Keylor NAVAS,Keilor Antonio,NAVAS GAMBOA,1986-12-15,CRC
1,255711,43901,Costa Rica,405824,2,midfielder,Daniel CHACON,Daniel Alonso,CHACÓN SALAS,2001-04-11,CRC
2,255711,43901,Costa Rica,411350,3,defender,Juan Pablo VARGAS,Juan Pablo,VARGAS CAMPOS,1995-06-06,CRC
3,255711,43901,Costa Rica,439979,4,defender,Keysher FULLER,Keysher,FULLER SPENCE,1994-07-12,CRC
4,255711,43901,Costa Rica,236530,5,midfielder,Celso BORGES,Celso,BORGES MORA,1988-05-27,CRC


In [25]:
df_minutos = df_pl[df_pl.team=='Spain'][['player_id', 'player_name']]

In [26]:
df_minutos['minutos_jugados'] = 0

### Cálculo de minutos jugados totales por jugador

In [27]:
minutos_jugados(df_esp_cri, df_minutos)
minutos_jugados(df_esp_ale, df_minutos)
minutos_jugados(df_esp_jap, df_minutos)
minutos_jugados(df_esp_mar, df_minutos)

,player_id,player_name,minutos_jugados
104,430736,Robert SANCHEZ,0
105,270948,Cesar AZPILICUETA,134
106,405562,Eric GARCIA,0
107,430721,Pau TORRES,90
108,303034,Sergio BUSQUETS,363
109,382739,Marcos LLORENTE,120
110,314291,Alvaro MORATA,185
111,313410,KOKE,52
112,447866,GAVI,284
113,400715,Marco ASENSIO,229


In [28]:
#df_minutos.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\viz\ficheros Tableau\tabla_minutos_jugados.xlsx', index=False)

In [29]:
df_esp_cri.from_player_name.unique()

array(['GAVI', 'AYMERIC LAPORTE', 'FRANCISCO CALVO', 'FERRAN TORRES',
       'RODRI', 'CESAR AZPILICUETA', 'JORDI ALBA', 'KEYSHER FULLER',
       'PEDRI', 'DANI OLMO', 'CARLOS MARTINEZ', 'ANTHONY CONTRERAS',
       'JEWISON BENNETTE', 'BRYAN OVIEDO', 'SERGIO BUSQUETS',
       'CELSO BORGES', 'MARCO ASENSIO', 'YELTSIN TEJEDA', 'JOEL CAMPBELL',
       'OSCAR DUARTE', 'UNAI SIMON', 'KEYLOR NAVAS', 'KENDALL WASTON',
       'ALVARO MORATA', 'CARLOS SOLER', 'BRYAN RUIZ', 'ALVARO ZAMORA',
       'ALEJANDRO BALDE', 'KOKE', 'NICO WILLIAMS', 'BRANDON AGUILERA',
       'RONALD MATARRITA'], dtype=object)

In [30]:
df_ev.head()

,match_id,match_run_time_in_ms,match_run_time,match_time_in_ms,event_id,player_seq_id,event_order,half_time,side,category,event,event_type,action_type,from_player_id,from_player_name,from_player_shirt_number,to_player_id,to_player_name,to_player_shirt_number,team_id,team_name,sequence_type,outcome,outcome_additional,opposition_touch,body_type,direction,pressure,style,style_additional,frame_location,game_state,origin,origin_additional,save_type,save_detail,stance,x_frame,y_frame,movement,offering_to_receive_total_units,line_break_direction,line_break_outcome,team_shape,team_unit,team_units_broken,total_team_units,event_end_time_in_ms,x,x_mirrored,y,y_mirrored,x_location_start,x_location_start_mirrored,x_location_end,x_location_end_mirrored,y_location_start,y_location_start_mirrored,y_location_end,y_location_end_mirrored,match_time_in_min
0,133004,0,00:00:00,0,1,1,1,1,NaN,in_possession,kickoff,start_restart,NaN,447866,GAVI,9,NaN,NaN,NaN,43969,SPAIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.499810,0.499810,0.503971,0.503971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,133004,0,00:00:00,0,2,1,2,1,NaN,in_possession,pass,distribution,NaN,447866,GAVI,9,335999.0,AYMERIC LAPORTE,24.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,right_foot,NaN,no_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.499810,0.499810,0.503971,0.503971,0.49981,0.49981,0.690286,0.309714,0.503971,0.503971,0.433382,0.566618,0
2,133004,1480,00:00:02,1480,3,2,1,1,r,in_possession,no_offer,offering_to_receive,NaN,335999,AYMERIC LAPORTE,24,NaN,NaN,NaN,43969,SPAIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_movement,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1480.0,0.691905,0.308095,0.428824,0.571176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,133004,1480,00:00:02,1480,4,2,2,1,r,in_possession,reception,receiving_possession,kickoff,335999,AYMERIC LAPORTE,24,NaN,NaN,NaN,43969,SPAIN,NaN,NaN,NaN,NaN,NaN,NaN,no_pressure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inside,in_front,NaN,3.0,NaN,0.691905,0.308095,0.428824,0.571176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,133004,1519,00:00:02,1519,5,1,3,1,r,in_possession,possession_outcome,possession_outcome,NaN,447866,GAVI,9,NaN,NaN,NaN,43969,SPAIN,possession,possession_complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.690286,0.309714,0.433382,0.566618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [31]:
df_ev[df_ev.event=='pass'].head()

,match_id,match_run_time_in_ms,match_run_time,match_time_in_ms,event_id,player_seq_id,event_order,half_time,side,category,event,event_type,action_type,from_player_id,from_player_name,from_player_shirt_number,to_player_id,to_player_name,to_player_shirt_number,team_id,team_name,sequence_type,outcome,outcome_additional,opposition_touch,body_type,direction,pressure,style,style_additional,frame_location,game_state,origin,origin_additional,save_type,save_detail,stance,x_frame,y_frame,movement,offering_to_receive_total_units,line_break_direction,line_break_outcome,team_shape,team_unit,team_units_broken,total_team_units,event_end_time_in_ms,x,x_mirrored,y,y_mirrored,x_location_start,x_location_start_mirrored,x_location_end,x_location_end_mirrored,y_location_start,y_location_start_mirrored,y_location_end,y_location_end_mirrored,match_time_in_min
1,133004,0,00:00:00,0,2,1,2,1,NaN,in_possession,pass,distribution,NaN,447866,GAVI,9,335999.0,AYMERIC LAPORTE,24.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,right_foot,NaN,no_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.499810,0.499810,0.503971,0.503971,0.499810,0.499810,0.690286,0.309714,0.503971,0.503971,0.433382,0.566618,0
5,133004,3779,00:00:04,3779,6,2,3,1,r,in_possession,pass,distribution,NaN,335999,AYMERIC LAPORTE,24,NaN,NaN,NaN,43969,SPAIN,NaN,possession_incomplete,defensive_event,NaN,left_foot,NaN,no_pressure,lofted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,over,line_break_incomplete,inside,NaN,1-2,3.0,NaN,0.646667,0.353333,0.413676,0.586324,0.646667,0.353333,0.177810,0.822190,0.413676,0.586324,0.828088,0.171912,0
15,133004,18409,00:00:19,18409,16,5,3,1,r,in_possession,pass,distribution,NaN,411375,RODRI,16,306954.0,JORDI ALBA,18.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,right_foot,NaN,no_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.644476,0.355524,0.801471,0.198529,0.644476,0.355524,0.674857,0.325143,0.801471,0.198529,0.508088,0.491912,0
20,133004,23710,00:00:24,23710,21,8,3,1,r,in_possession,pass,distribution,NaN,335999,AYMERIC LAPORTE,24,430750.0,DANI OLMO,21.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,right_foot,NaN,indirect_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.582476,0.417524,0.400882,0.599118,0.582476,0.417524,0.514952,0.485048,0.400882,0.599118,0.167353,0.832647,0
27,133004,26289,00:00:27,26289,28,7,3,1,r,in_possession,pass,distribution,NaN,306954,JORDI ALBA,18,335999.0,AYMERIC LAPORTE,24.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,left_foot,NaN,no_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,around,line_break_complete,outside,NaN,1,3.0,NaN,0.482000,0.518000,0.119118,0.880882,0.482000,0.518000,0.363238,0.636762,0.119118,0.880882,0.026618,0.973382,0


In [32]:
df_ev[(df_ev.event=='pass') & (df_ev.from_player_name==df_ev.to_player_name)]

,match_id,match_run_time_in_ms,match_run_time,match_time_in_ms,event_id,player_seq_id,event_order,half_time,side,category,event,event_type,action_type,from_player_id,from_player_name,from_player_shirt_number,to_player_id,to_player_name,to_player_shirt_number,team_id,team_name,sequence_type,outcome,outcome_additional,opposition_touch,body_type,direction,pressure,style,style_additional,frame_location,game_state,origin,origin_additional,save_type,save_detail,stance,x_frame,y_frame,movement,offering_to_receive_total_units,line_break_direction,line_break_outcome,team_shape,team_unit,team_units_broken,total_team_units,event_end_time_in_ms,x,x_mirrored,y,y_mirrored,x_location_start,x_location_start_mirrored,x_location_end,x_location_end_mirrored,y_location_start,y_location_start_mirrored,y_location_end,y_location_end_mirrored,match_time_in_min
970,133004,539060,00:08:59,539060,971,342,3,1,r,in_possession,pass,distribution,NaN,335999,AYMERIC LAPORTE,24,335999.0,AYMERIC LAPORTE,24.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,left_foot,NaN,no_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.535429,0.464571,0.175735,0.824265,0.535429,0.464571,0.600952,0.399048,0.175735,0.824265,0.463382,0.536618,8
1060,133004,579589,00:09:40,579589,1061,371,3,1,r,in_possession,pass,distribution,NaN,405545,FERRAN TORRES,11,405545.0,FERRAN TORRES,11.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,right_foot,NaN,indirect_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.174381,0.825619,0.879118,0.120882,0.174381,0.825619,0.142762,0.857238,0.879118,0.120882,0.986912,0.013088,9
1284,133004,646900,00:10:47,646900,1285,450,3,1,r,in_possession,pass,distribution,NaN,430750,DANI OLMO,21,430750.0,DANI OLMO,21.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,left_foot,NaN,indirect_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.197238,0.802762,0.272353,0.727647,0.197238,0.802762,0.189619,0.810381,0.272353,0.727647,0.437206,0.562794,10
1369,133004,738720,00:12:19,738720,1370,483,3,1,r,in_possession,pass,distribution,NaN,335999,AYMERIC LAPORTE,24,335999.0,AYMERIC LAPORTE,24.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,left_foot,NaN,indirect_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.918952,0.081048,0.053676,0.946324,0.918952,0.081048,0.927619,0.072381,0.053676,0.946324,0.391618,0.608382,12
1742,133004,1000599,00:16:41,1000599,1743,614,4,1,r,in_possession,pass,distribution,NaN,411375,RODRI,16,411375.0,RODRI,16.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,left_foot,NaN,no_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,around,line_break_complete,outside,NaN,1,3.0,NaN,0.591619,0.408381,0.663971,0.336029,0.591619,0.408381,0.505905,0.494095,0.663971,0.336029,0.903382,0.096618,16
1846,133004,1056430,00:17:37,1056430,1847,651,2,1,l,in_possession,pass,distribution,NaN,312987,OSCAR DUARTE,6,312987.0,OSCAR DUARTE,6.0,43901,COSTA RICA,NaN,possession_complete,NaN,NaN,right_foot,NaN,no_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.276857,0.276857,0.254412,0.254412,0.276857,0.276857,0.116952,0.116952,0.254412,0.254412,0.462794,0.462794,17
2507,133004,1449650,00:24:10,1449650,2508,887,3,1,r,in_possession,pass,distribution,NaN,335999,AYMERIC LAPORTE,24,335999.0,AYMERIC LAPORTE,24.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,right_foot,NaN,no_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_line_break,NaN,NaN,NaN,NaN,NaN,0.673714,0.326286,0.397206,0.602794,0.673714,0.326286,0.594286,0.405714,0.397206,0.602794,0.187941,0.812059,24
2778,133004,1545139,00:25:45,1545139,2779,977,3,1,r,in_possession,pass,distribution,NaN,270948,CESAR AZPILICUETA,2,270948.0,CESAR AZPILICUETA,2.0,43969,SPAIN,NaN,possession_complete,NaN,NaN,right_foot,NaN,indirect_pressure,push,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [33]:
len(df_ev[(df_ev.event=='pass') & (df_ev.from_player_name==df_ev.to_player_name)])

145